# Mindnlp PEFT with DNA Language Models
This notebook demonstrates how to utilize parameter-efficient fine-tuning techniques (PEFT) from the PEFT library to fine-tune a DNA Language Model (DNA-LM). The fine-tuned DNA-LM will be applied to solve a task from the nucleotide benchmark dataset. Parameter-efficient fine-tuning (PEFT) techniques are crucial for adapting large pre-trained models to specific tasks with limited computational resources.

## 1. Import relevant libraries
We'll start by importing the required libraries, including the mindnlp library and other dependencies.

In [1]:
import mindspore
import mindnlp
import numpy as np
import tqdm

/home/daiyuxin/miniconda3/envs/neoming_vivit/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.634 seconds.
Prefix dict has been built successfully.


## 2. Load models
We'll load a pre-trained DNA Language Model, "SpeciesLM", that serves as the base for fine-tuning. This is done using the transformers library from HuggingFace.

The tokenizer and the model comes from the paper, "Species-aware DNA language models capture regulatory elements and their evolution". Paper Link, Code Link. They introduce a species-aware DNA language model, which is trained on more than 800 species spanning over 500 million years of evolution.

In [2]:
from mindnlp.transformers import AutoTokenizer, AutoModelForMaskedLM

In [3]:
tokenizer = AutoTokenizer.from_pretrained("gagneurlab/SpeciesLM", revision = "downstream_species_lm")
lm = AutoModelForMaskedLM.from_pretrained("gagneurlab/SpeciesLM", revision = "downstream_species_lm")

/home/daiyuxin/miniconda3/envs/neoming_vivit/lib/python3.9/site-packages/mindnlp/transformers/tokenization_utils_base.py:1526: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted, and will be then set to `False` by default. 
  warnings.warn(


In [4]:
lm.eval()

BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(5504, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear (768 -> 768)
              (key): Linear (768 -> 768)
              (value): Linear (768 -> 768)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear (768 -> 768)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
          )
          (intermediate): BertIntermedia

## 3. Prepare datasets
We'll load the `nucleotide_transformer_downstream_tasks` dataset, which contains 18 downstream tasks from the Nucleotide Transformer paper. This dataset provides a consistent genomics benchmark with binary classification tasks.

In [5]:
from mindnlp.dataset import load_dataset
raw_data = load_dataset("InstaDeepAI/nucleotide_transformer_downstream_tasks", "H3")
raw_data

{'train': <mindspore.dataset.engine.datasets_user_defined.GeneratorDataset at 0x7f6ee71d69a0>,
 'test': <mindspore.dataset.engine.datasets_user_defined.GeneratorDataset at 0x7f6ee7470a30>}

We'll use the "H3" subset of this dataset, which contains a total of 13,468 rows in the training data, and 1497 rows in the test data.

In [6]:
train_dataset = raw_data['train']
test_dataset = raw_data['test']

def show_dataset_info(dataset):
    print("dataset column: {}".format(dataset.get_col_names()))
    print("dataset size: {}".format(dataset.get_dataset_size()))
    print("dataset batch size: {}\n".format(dataset.get_batch_size()))

print("train dataset info:")
show_dataset_info(train_dataset)
print("test dataset info:")
show_dataset_info(test_dataset)

train dataset info:
dataset column: ['sequence', 'name', 'label']
dataset size: 13468
dataset batch size: 1

test dataset info:
dataset column: ['sequence', 'name', 'label']
dataset size: 1497
dataset batch size: 1



The dataset consists of three columns, `sequence`, `name` and `label`. An row in this dataset looks like:

In [7]:
for data in train_dataset:
    print(data)
    break

[Tensor(shape=[], dtype=String, value= 'TCACTTCGATTATTGAGGCAGTCTTCATTAAAGTTTATTACAATGGATATGGTATCACCAGTCTTGAACCTACAATCATCTATTTTAGGTGAGCTCGTAGGCATTATTGGAAAAGTGTTCTTTCTCTTAATAGAAGAGATTAAATACCCGATAATCACACCCAAAATTATTGTGGATGCCCAGATATCTTCTTGGTCATTGTTTTTTTTCGCTTCAATCTGTAATCTCTCTGCAAAATTTCGGGAGCCAATAGTGACAACATCGTCAATAATAAGTTTGATGGAATCGGAAAAAGATCTTAAAAATGTAAATGAGTATTTCCAAATAATGGCCAAAATGCTCTTTATATTGGAAAATAAAATAGTTGTTTCGCTCTTCGTAGTATTTAACATTTCCGTTCTTATCATTGTAAAGTCTGAGCCATATTCATATGGAAAAGTGCTTTTTAAACCTAGTTCCTCCATATTTTAGTTTTTTATCGATATTGGAAAAAAAAGAGC'), Tensor(shape=[], dtype=String, value= 'YBR063C_YBR063C_367930|0'), Tensor(shape=[], dtype=Int64, value= 0)]


We split out dataset into training, test, and validation sets.

In [8]:
train_dataset, valid_dataset = train_dataset.split([0.85, 0.15])

print("train dataset info:")
show_dataset_info(train_dataset)
print("valid dataset info:")
show_dataset_info(valid_dataset)

train dataset info:
dataset column: ['sequence', 'name', 'label']
dataset size: 11448
dataset batch size: 1

valid dataset info:
dataset column: ['sequence', 'name', 'label']
dataset size: 2020
dataset batch size: 1



Then, we use the tokenizer and a utility function we created, `get_kmers` to generate the final data and labels. The `get_kmers` function is essential for generating overlapping 6-mers needed by the language model (LM). By using k=6 and stride=1, we ensure that the model receives continuous and overlapping subsequences, capturing the local context within the biological sequence for more effective analysis and prediction.

In [9]:
def get_kmers(seq, k=6, stride=1):
    return [seq[i:i + k] for i in range(0, len(seq), stride) if i + k <= len(seq)]

In [10]:

BATCH_SIZE = 16
DATASET_LIMIT = 200 # NOTE: This dataset limit is set to 200, so that the training runs faster. It can be set to None to use the
                    # entire dataset

def process_sequence(data):
    sequence = data.tolist()
    sequence = "candida_glabrata " + " ".join(get_kmers(sequence))
    sequence = tokenizer(sequence)["input_ids"]
    return sequence

def nucleotide_dataset_process(dataset):
    # remove name column
    dataset = dataset.project(columns=['sequence', 'label'])

    # process sequence
    dataset = dataset.map(process_sequence, input_columns=['sequence'], output_columns=['sequence'])

    # change dataset size
    dataset = dataset.take(DATASET_LIMIT)

    # batch with padding
    dataset = dataset.padded_batch(batch_size=BATCH_SIZE,
                                   drop_remainder=True,
                                   pad_info={'sequence':(None, -100)})
    return dataset


Finally, we create a Dataset object for each our sets.

In [11]:
train_dataset = train_dataset.apply(nucleotide_dataset_process)
valid_dataset = valid_dataset.apply(nucleotide_dataset_process)
test_dataset = test_dataset.apply(nucleotide_dataset_process)

print("train dataset info:")
show_dataset_info(train_dataset)

print("valid dataset info:")
show_dataset_info(valid_dataset)

print("test dataset info:")
show_dataset_info(test_dataset)

for data in train_dataset:
    print(data)
    break

train dataset info:
dataset column: ['sequence', 'label']
dataset size: 12
dataset batch size: 16

valid dataset info:
dataset column: ['sequence', 'label']
dataset size: 12
dataset batch size: 16

test dataset info:
dataset column: ['sequence', 'label']
dataset size: 12
dataset batch size: 16

[Tensor(shape=[16, 498], dtype=Int64, value=
[[   2, 4724, 2162 ... 1240,  852,    3],
 [   2, 4724, 3690 ... 2394, 1369,    3],
 [   2, 4724,  665 ... 1726, 2794,    3],
 ...
 [   2, 4724, 3446 ...  860, 3427,    3],
 [   2, 4724, 2966 ... 1525, 1992,    3],
 [   2, 4724, 3297 ... 1970, 3770,    3]]), Tensor(shape=[16], dtype=Int64, value= [1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0])]


## 4. Train model
Now, we'll train our DNA Language Model with the training dataset. We'll add a linear layer in the final layer of our language model, and then, train all the parameteres of our model with the training dataset.

In [12]:
import mindspore
from mindnlp.core import nn

class DNA_LM(nn.Module):
    def __init__(self, model, num_labels):
        super(DNA_LM, self).__init__()
        self.model = model.bert
        self.in_features = model.config.hidden_size
        self.out_features = num_labels
        self.classifier = nn.Linear(self.in_features, self.out_features)

    def forward(self, sequence, label=None):
        outputs = self.model(input_ids=sequence, attention_mask=None, output_hidden_states=True)
        sequence_output = outputs.hidden_states[-1]
        # Use the [CLS] token for classification
        cls_output = sequence_output[:, 0, :]
        logits = self.classifier(cls_output)

        loss = None
        if label is not None:
            loss_fct = nn.CrossEntropyLoss()
            loss = loss_fct(logits.view(-1, self.out_features), label.view(-1))

        return (loss, logits) if loss is not None else logits

# Number of classes for your classification task
num_labels = 2
classification_model = DNA_LM(lm, num_labels)
classification_model

DNA_LM(
  (model): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(5504, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear (768 -> 768)
              (key): Linear (768 -> 768)
              (value): Linear (768 -> 768)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear (768 -> 768)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
          )
          (intermediate): BertIntermediate(
    

In [13]:
from mindnlp.engine import Trainer, TrainingArguments

# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    num_train_epochs=5,
    weight_decay=0.01,
    eval_steps=1,
    logging_steps=1,
)

# Initialize Trainer
trainer = Trainer(
    model=classification_model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset,
    tokenizer=tokenizer,
)

# Train the model
trainer.train()

  2%|▏         | 1/60 [00:01<01:36,  1.64s/it]

{'loss': 0.8075, 'learning_rate': 1.9666666666666666e-05, 'epoch': 0.08}


  3%|▎         | 2/60 [00:02<01:10,  1.22s/it]

{'loss': 0.8228, 'learning_rate': 1.9333333333333333e-05, 'epoch': 0.17}


  5%|▌         | 3/60 [00:03<00:57,  1.01s/it]

...

100%|██████████| 60/60 [00:58<00:00,  1.26it/s]

{'loss': 0.3635, 'learning_rate': 0.0, 'epoch': 5.0}


                                               
100%|██████████| 60/60 [01:00<00:00,  1.01s/it]

{'eval_loss': 0.4767491817474365, 'eval_runtime': 2.2767, 'eval_samples_per_second': 5.271, 'eval_steps_per_second': 0.878, 'epoch': 5.0}
{'train_runtime': 60.4651, 'train_samples_per_second': 15.877, 'train_steps_per_second': 0.992, 'train_loss': 0.5313406201700369, 'epoch': 5.0}


TrainOutput(global_step=60, training_loss=0.5313406201700369, metrics={'train_runtime': 60.4651, 'train_samples_per_second': 15.877, 'train_steps_per_second': 0.992, 'train_loss': 0.5313406201700369, 'epoch': 5.0})